In [1]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2
import dlib
import argparse
from imutils.video import VideoStream
from imutils import face_utils, translate, rotate, resize

In [2]:
vs = VideoStream().start()

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
glasses = Image.open("sunglasses.png")
beach = Image.open("beach.jpg")

max_width = 540
current_fps = 0
KERNAL_SIZE = 21
CANNY_THRESH_1 = 10
CANNY_THRESH_2 = 200
MASK_DILATE_ITER = 10
MASK_ERODE_ITER = 10

wear_glasses = False
remove_background = False

while True:
    frame = vs.read()
    frame = resize(frame, width=max_width)
    
    current_fps = (current_fps+1)%3 # apply face detection every 3 frame.
    
    if wear_glasses == True:
        if current_fps==0:
            frame_RGB = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            rects = detector(frame_gray, 0)
            glasses_list = []

            for rect in rects:
                shades_width = rect.right() - rect.left()
                
                # predictor used to detect orientation in place where current face is
                shape = predictor(frame_gray, rect)
                shape = face_utils.shape_to_np(shape)
                
                # grab the outlines of each eye
                leftEye = shape[36:42]
                rightEye = shape[42:48]
                
                # compute the center of mass for each eye
                leftEyeCenter = leftEye.mean(axis=0).astype("int")
                rightEyeCenter = rightEye.mean(axis=0).astype("int")
                
                # compute the angle between the eye centroids
                dY = leftEyeCenter[1] - rightEyeCenter[1] 
                dX = leftEyeCenter[0] - rightEyeCenter[0]
                angle = np.rad2deg(np.arctan2(dY, dX)) 
                
                # modify the sun glasses image to fit the faces
                sun_glasses = glasses.resize((shades_width, int(shades_width * glasses.size[1] / glasses.size[0])),
                                             resample=Image.LANCZOS)
                sun_glasses = sun_glasses.rotate(angle, expand=True)
                sun_glasses = sun_glasses.transpose(Image.FLIP_TOP_BOTTOM)
                
                x = leftEye[0,0] - shades_width // 4
                y = leftEye[0,1] - shades_width // 6
                glasses_list.append((sun_glasses, x, y))
        
        # paste the sun glasses for each face
        try:
            for i in glasses_list:
                frame_RGB.paste(i[0], (i[1], i[2]), i[0])
                frame = cv2.cvtColor(np.asarray(frame_RGB), cv2.COLOR_RGB2BGR)
        except:
            pass
        
    if remove_background == True:
        if current_fps==0: 
            # Another way is to use MOG2 to isolates moving items from static background, with the following code:
            #frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            #fgmask = cv2.createBackgroundSubtractorMOG2().apply(frame_gray)
            #fgmask[np.nonzero(fgmask)] = 1
            #frame = np.dstack([fgmask]*3) * frame
            
            # Canny edge detection
            edges = cv2.Canny(frame, CANNY_THRESH_1, CANNY_THRESH_2)
            edges = cv2.dilate(edges, None)
            edges = cv2.erode(edges, None)

            # Find contours in edges, sort by area
            contour_info = []
            contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            # For a higher version of cv2, this line should be:
            # _, contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            for c in contours:
                contour_info.append((
                    c,
                    cv2.isContourConvex(c),
                    cv2.contourArea(c),
                ))
            contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
            
            try:
                # Create empty mask, draw filled polygon on it corresponding to largest contour
                max_contour = contour_info[0]
                mask = np.zeros(edges.shape)
                cv2.fillConvexPoly(mask, max_contour[0], (255))

                # Smooth mask, then blur it
                mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
                mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
                mask = cv2.GaussianBlur(mask, (KERNAL_SIZE, KERNAL_SIZE), 0)
                mask_stack = np.dstack([mask]*3).astype('float32')/255.0
            except:
                pass
        
        # Blend masked img into MASK_COLOR background
        try:
            bg = beach.resize((frame.shape[1], frame.shape[0]), resample=Image.LANCZOS)
            bg = cv2.cvtColor(np.asarray(bg), cv2.COLOR_RGB2BGR)
            #frame = frame.astype('float32')/255.0
            frame = (mask_stack * frame) + ((1-mask_stack) * bg)
            frame = frame.astype('uint8')
        except:
            pass

    cv2.imshow("cool vedio", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

    if key == ord("g"):
        wear_glasses = not wear_glasses
        
    if key == ord("r"):
        remove_background = not remove_background

cv2.destroyAllWindows()
vs.stop()